In [ ]:
import os
import numpy as np
import scipy.interpolate as sci
import matplotlib.pyplot as plt
import json
from astropy.time import Time as astrotime

import tdepps.model_injection as Inj
from tdepps.model_toolkit import (create_run_dict, SignalFluenceInjector,
                                  make_rate_records, rebin_rate_rec,
                                  UniformTimeSampler, SinusFixedConstRateFunction)

secinday = 24. * 60. * 60.
rndgen = np.random.RandomState(42439462)
print("Loaded: ", astrotime.now())

tw_id = 20

## Setup

In [ ]:
# ###### Load data for each sample
p = "/Users/tmenne/git/phd/hese_tdepps/data/proc"
exp = {}
names = ["79", "86I", "86II", "86III"]
for key in names[:1]:
    exp[key] = np.load(os.path.join(p, "offdata/{}.npy".format(key)))
    print("Loaded exp {}.".format(os.path.join(p,
                                               "offdata/{}.npy".format(key))))

exp_names = {key: exp_i.dtype.names for key, exp_i in exp.items()}

p = "/Users/tmenne/git/phd/hese_tdepps/data/raw"
mc = {}
mc["79"] = np.load(os.path.join(p, "IC79/IC79_corrected_MC.npy"))
print("Loaded mc from: {}".format(
    os.path.join(p, "IC79/IC79_corrected_MC.npy")))
# mc["86I"] = np.load(os.path.join(p, "IC86_2011/IC86_corrected_MC.npy"))
# print("Loaded mc from: {}".format(
#     os.path.join(p, "IC79/IC79_corrected_MC.npy")))
# mc["86II"] = np.load(os.path.join(p, "IC86_2012/IC86-2012_corrected_MC.npy"))
# mc["86III"] = mc["86II"]  # MC is the same here
# print("Loaded mc from: {} for 86II & 86III".format(
#     os.path.join(p, "IC86_2012/IC86-2012_corrected_MC.npy")))

# We need "timeMJD" as a key, so change the dtype names
for mci in mc.values():
    if "timeMJD" not in mci.dtype.names:
        idx = mci.dtype.names.index("time")
        mci.dtype.names = (mci.dtype.names[:idx] + ("timeMJD",) +
                           mci.dtype.names[idx + 1:])


# ###### Get prepared run time lists
p = "/Users/tmenne/git/phd/hese_tdepps/data/proc"
rundict = {}

rundict["79"] = json.load(open(os.path.join(p, "goodruns/IC79v24.json")))
# rundict["86I"] = json.load(open(os.path.join(p, "goodruns/IC86_2011.json")))
# rundict["86II"] = json.load(open(os.path.join(p, "goodruns/IC86_2012.json")))
# rundict["86III"] = json.load(open(os.path.join(p, "goodruns/IC86_2013.json")))

print("Loaded goodrun dicts from: " + p + "/goddruns")


# ###### Load HESE tracks locations
p = "/Users/tmenne/git/phd/hese_tdepps/data/raw"
path = os.path.join(p, "public_data_release/All_HESE_Events_4_years_tracks.txt")
src_t, src_dec, src_ra = np.loadtxt(path, usecols=[1, 2, 3], unpack=True)
src_ra = np.deg2rad(src_ra)
src_dec = np.deg2rad(src_dec)

names = ["t", "dt0", "dt1", "ra", "dec", "w_theo"]
types = len(names) * [np.float]
dtype = [(name, typ) for name, typ in zip(names, types)]

srcs = {}
for key, rl in rundict.items():
    runs = rl["runs"]
    tmin = np.amin([astrotime(r["good_tstart"]).mjd for r in runs])
    tmax = np.amax([astrotime(r["good_tstop"]).mjd for r in runs])
    t_mask = (src_t >= tmin) & (src_t <= tmax)

    src_ti = src_t[t_mask]
    nsrcs_i = len(src_ti)

    srcs_i = np.empty((nsrcs_i, ), dtype=dtype)

    srcs_i["t"] = src_ti
    # Leave empty for now, time window is set explicitely below
    srcs_i["dt0"] = np.zeros(nsrcs_i, dtype=np.float)
    srcs_i["dt1"] = np.zeros(nsrcs_i, dtype=np.float)
    srcs_i["ra"] = src_ra[t_mask]
    srcs_i["dec"] = src_dec[t_mask]
    srcs_i["w_theo"] = np.ones(nsrcs_i, dtype=np.float)
    srcs[key] = srcs_i

print("Made HESE srcs from: " + path)


# ###### Setup time window for all srcs
fname = "/Users/tmenne/git/phd/hese_tdepps/data/proc/time_windows/time_windows.txt"
time_window = np.loadtxt(fname=fname)[tw_id]
print("Using time window {:2d}: {:.2f}s".format(tw_id, np.diff(time_window)[0]))
for key, srcs_i in srcs.items():
    srcs_i["dt0"] = np.repeat(time_window[0], repeats=len(srcs_i))
    srcs_i["dt1"] = np.repeat(time_window[1], repeats=len(srcs_i))
    print(srcs_i)

## Done

In [ ]:
print("Done!")

## Start

In [ ]:
IC79 = exp["79"]
IC79MC = mc["79"]
IC79srcs = srcs["79"]
IC79rd = create_run_dict(rundict["79"]["runs"])

hor = 0.25
sindec_bins = np.unique(np.concatenate([
                        np.linspace(-1., -hor, 5 + 1),    # south
                        np.linspace(-hor, +hor, 10 + 1),  # horizon
                        np.linspace(+hor, 1., 5 + 1),     # north
                        ]))
rate_rebins = np.linspace(IC79["timeMJD"].min(), IC79["timeMJD"].max(), 12)

bg_inj_args = {"sindec_bins": sindec_bins, "rate_rebins": rate_rebins}
grb_inj = Inj.GRBModelInjector(bg_inj_args=bg_inj_args, rndgen=rndgen)

In [ ]:
recs = make_rate_records(T=IC79["timeMJD"], run_dict=IC79rd)
rates, new_bins, stddev, _ = rebin_rate_rec(rate_rec=recs, bins=rate_rebins,
                                     ignore_zero_runs=True)
mids = 0.5 * (new_bins[:-1] + new_bins[1:])

rf = SinusFixedConstRateFunction(p_fix=365.)
allsky_res = rf.fit(rate=rates, srcs=IC79srcs, t=mids, w=1. / stddev)

t0_fix = allsky_res.x[1]
print("Best fit t0 before first event: ",
      t0_fix - IC79["timeMJD"].min(), "days")

rf = SinusFixedConstRateFunction(p_fix=365., t0_fix=t0_fix)

In [ ]:
def scan_llh(mids, rates, stddev, bf_a, bf_d, a_rng, d_rng, rf, nbins=100):
    a_bins = np.linspace(bf_a - a_rng, bf_a + a_rng, nbins)
    d_bins = np.linspace(bf_d - d_rng, bf_d + d_rng, nbins)
    
    a, d = np.meshgrid(a_bins, d_bins)
    AA, DD = map(np.ravel, [a, d])

    llh = np.empty_like(AA)
    for i, (ai, di) in enumerate(zip(AA, DD)):
        llh[i] = rf._lstsq((ai, di), mids, rates, 1. / stddev)
        
    llh = llh.reshape(a.shape)
    
    plt.pcolormesh(a, d, llh)
    plt.colorbar()

    vals = np.amin(llh) - scs.chi2.logsf(df=2, x=[1**2, 2**2, 3**2])
    cs = plt.contour(a, d, llh, vals, linestyles=["-", "--", ":"], colors="w")
    # https://stackoverflow.com/questions/5666056
    paths = cs.collections[0].get_paths()[0]
    v = paths.vertices
    x = v[:, 0]
    y = v[:, 1]
    
    a_min, a_max = np.amin(x), np.amax(x)
    d_min, d_max = np.amin(y), np.amax(y)
    std_a, std_d = 0.5 * (a_max - a_min), 0.5 * (d_max - d_min)
    
    plt.plot(bf_a, bf_d, ls="", marker="o", c="w")
    
    plt.plot([a_min, a_max], [bf_d, bf_d], c="w", ls="--")
    plt.plot([bf_a, bf_a], [d_min, d_max], c="w", ls="--")
    plt.plot([bf_a - std_a, bf_a + std_a], [bf_d, bf_d], c="r", ls=":")
    plt.plot([bf_a, bf_a], [bf_d - std_d, bf_d + std_d], c="r", ls=":")
    
    plt.xlabel = ("a")
    plt.ylabel = ("d")
    plt.show()
    
    return std_a, std_d

In [ ]:
sindec = IC79["sinDec"]
t_ = np.linspace(IC79["timeMJD"].min(), IC79["timeMJD"].max(), 200)

allres = []
errs = []
for j, (lo, hi) in enumerate(zip(sindec_bins[:-1], sindec_bins[1:])):
    mask = (sindec >= lo) & (sindec <= hi)

    recs = make_rate_records(T=IC79["timeMJD"][mask], run_dict=IC79rd)
    rates, new_bins, stddev, _ = rebin_rate_rec(rate_rec=recs, bins=rate_rebins,
                                                ignore_zero_runs=True)
    new_mids = 0.5 * (new_bins[:-1] + new_bins[1:])
    
    res = rf.fit(rate=rates, srcs=IC79srcs, t=new_mids, w=1. / stddev)
    allres.append(res)
    
    plt.errorbar(recs["start_mjd"], recs["rate"], yerr=recs["rate_std"],
                 fmt=",", alpha=0.2, color="C0")
    plt.plot(recs["start_mjd"], recs["rate"], marker=".", ls="", color="C0")
    plt.plot(new_bins, np.r_[rates[0], rates], drawstyle="steps-pre", color="k")
    plt.plot(t_, rf.fun(t=t_, pars=res.x), color="C3")

    plt.show()
    
    std_a, std_d = scan_llh(new_mids, rates, stddev, res.x[0], res.x[1],
                            a_rng=1e-5, d_rng=5e-6, rf=rf)
    errs.append([std_a, std_d])

errs = np.array(errs)

In [ ]:
for res in allres:
    print("#########")
    print(res.nfev)
    print(res.hess_inv.todense())
    print(res.message)

In [ ]:
idx = 0
mids = 0.5 * (sindec_bins[:-1] + sindec_bins[1:])
norm = np.diff(sindec_bins)

for i, res in enumerate(allres):
    plt.errorbar(mids[i], res.x[idx] / norm[i], fmt=".", color="C7",
#                  yerr=None)
                 yerr=np.sqrt(res.hess_inv.todense()[idx, idx]) / norm[i])
    
plt.show()

In [ ]:
def signal_flux_model(trueE, E0=1., gamma=2.):
    return (trueE / E0)**(-gamma)

ts = UniformTimeSampler(random_state=None)

sig_inj = SignalFluenceInjector(signal_flux_model, time_sampler=ts)
sig_inj.fit(IC79srcs, MC=IC79MC, exp_names=["timeMJD", "dec", "ra", "sigma", "logE"])

In [ ]:
grb_inj.fit(X=IC79, srcs=IC79srcs, run_dict=IC79rd, sig_inj=sig_inj)

In [ ]:
sam = grb_inj.get_sample()

In [ ]:
n = "base"

bins = grb_inj._sin_dec_bins
mids = 0.5 * (bins[:-1] + bins[1:])
vals = grb_inj._best_pars[n]
# Errors larger than param > scale down for meaningful spline fit
inv = (np.abs(grb_inj._best_pars[n]) < grb_inj._best_stddevs[n])
# inv = np.zeros_like(vals, dtype=bool)
err = np.copy(grb_inj._best_stddevs[n])
err[inv] = np.amax(err[~inv])

plt.plot(mids, grb_inj._best_pars[n], color="k", ls="--")
plt.errorbar(mids[~inv], vals[~inv], yerr=err[~inv], fmt=",", color="k")
plt.errorbar(mids[inv], vals[inv], yerr=err[inv], fmt="o", color="C0")
plt.plot(x, grb_inj._param_splines[n](x), color="C7")

plt.show()

In [ ]:
x = np.linspace(-1, 1, 200)
bins = grb_inj._sin_dec_bins
# bins = np.linspace(-1, 1, 17 + 1)
for j, ev in enumerate(grb_inj._bg_inj):
    plt.hist(np.sin(IC79["dec"]), bins=bins, density=True)
    plt.plot(x, grb_inj._data_spl(x))

    plt.hist(np.sin(ev["dec"]), bins=bins, density=True, histtype="step", lw=2.5)
    plt.plot(x, grb_inj._sin_dec_splines[j](x))

    plt.show()
    